In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import norm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from data.generate_data import generate_data

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Hyperparameters

In [2]:
seed = 0
np.random.seed(seed=seed)

N = 100000        # the size of the unlabeled dataset
n = 5000            # pilot sample size
alpha = 1 # n**(-0.1)  
print(f"alpha={alpha:.4f}")

p = 25          # feature dimension
K = 2           # (K+1) classes
M = 200          # the size of the annotator pool
print(f"[n*alpha={int(n*alpha)}/N={N}]")
print(f"[M={M}] vs [n*alpha={int(n*alpha)}]")

alpha=1.0000
[n*alpha=5000/N=100000]
[M=200] vs [n*alpha=5000]


# Data Generation

In [3]:
beta, sigma_list, theta, X, Y, X1, X2, Y1, Y2, A1, AY1 = generate_data(K, p, N, n, M, alpha, seed=0)

True Labels 0    36979
1    32505
2    30516
dtype: int64 

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 1. 0. 1. 1. 1. 1. 0. 2. 0. 1. 1. 0.
  1. 0. 0. 0. 1. 0. 2. 2. 2. 0. 0. 1. 0. 0. 0. 2. 0. 1. 1. 1. 0. 1. 1. 1.
  2. 0. 0. 1. 1. 1. 2. 0. 1. 2. 1. 2. 2. 1. 2. 0. 2. 0. 0. 2. 1. 0. 1. 0.
  2. 1. 2. 0. 2. 0. 1. 0. 1. 0. 1. 2. 1. 2. 2. 1. 2. 0. 0. 0. 2. 0. 2. 2.
  2. 2. 0. 1. 1. 0. 0. 1. 2. 0. 1. 2. 2. 2. 1. 1. 2. 0. 1. 2. 0. 2. 1. 0.
  1. 1. 2. 0. 2. 2. 1. 2. 2. 2. 0. 2. 1. 2. 0. 2. 2. 0. 1. 0. 1. 1. 1. 1.
  0. 2. 1. 0. 2. 0. 0. 1. 2. 1. 2. 2. 0. 0. 1. 2. 0. 2. 0. 1. 1. 0. 0. 2.
  2. 0. 0. 2. 2. 0. 0. 2. 2. 0. 0. 0. 0. 1. 2. 1. 2. 1. 0. 1. 0. 2. 1. 2.
  2. 1. 2. 0. 0. 2. 0. 2.]
 [2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 1. 2. 2. 2. 0. 1. 2. 0. 2. 1. 2. 1. 0.
  0. 2. 2. 0. 1. 2. 0. 2. 2. 1. 2. 1. 2. 1. 0. 2. 0. 2. 0. 0. 1. 2. 1. 1.
  1. 1. 2. 2. 2. 2. 0. 2. 0. 0. 2. 1. 1. 1. 1. 0. 2. 0. 1. 0. 1. 0. 2. 2.
  1. 2. 1. 1. 2. 1. 2. 0. 2. 2. 1. 2. 1. 0. 0. 1. 0. 2. 1. 0. 2. 2. 2. 1.
  2. 1. 0. 2. 0. 0. 1. 1.

# Initial Estimator

In [4]:
from sklearn.linear_model import LogisticRegression

initial_b = np.zeros((M, K+1, p))
initial_beta = np.zeros((K+1, p))
initial_sigma = np.zeros(M)

In [5]:
for m in range(M):
    y_m = AY1[:, m]
    idx = (A1[:, m] != 0)
    X_m = X1[idx]
    y_m = y_m[idx]
    clf = LogisticRegression(random_state=0, fit_intercept=False).fit(X_m, y_m)
    initial_b[m] = clf.coef_
    initial_b[m] -= initial_b[m, 0]
    initial_sigma[m] = 1 / norm(initial_b[m])
    initial_beta += initial_b[m] * initial_sigma[m] / M

In [6]:
initial_theta = np.zeros(p * (K+1) + M)
initial_theta[:(p * (K+1))] = initial_beta.ravel()
initial_theta[(p * (K+1)):] = initial_sigma

In [7]:
norm(initial_beta - beta), norm(initial_sigma - sigma_list)

(0.3424202099798993, 42.70290552011125)

In [8]:
star_idx = 90
end_idx = 100
initial_sigma[star_idx:end_idx], sigma_list[star_idx:end_idx]

(array([2.90066357, 2.78924245, 3.36764056, 2.98284874, 3.80218463,
        3.3653014 , 3.02257884, 3.17210824, 3.39102676, 3.4233943 ]),
 array([4.6 , 4.65, 4.7 , 4.75, 4.8 , 4.85, 4.9 , 4.95, 5.  , 5.05]))

# One Step Update

In [9]:
from model.OS_Update import OS

In [10]:
initial_beta[1:,].shape

(2, 25)

In [11]:
os_model = OS(X1, AY1, A1, K, initial_beta[1:,], initial_sigma)
os_beta, os_sigma = os_model.one_step_update()

[1st derivative] beta
[1st derivative] sigma
[2st derivative] (j, k)
[2st derivative] (j, k)
[2st derivative] (j, k)
[2st derivative] (j, k)


In [12]:
norm(os_beta - beta[1:].ravel()), norm(os_sigma - sigma_list)

(0.15294440412649957, 41.39192425148104)

# MLE

In [13]:
from model.MLE import MLE

In [17]:
mle_model = MLE(X1, AY1, A1, K)

In [18]:
mle_model.GD_alg(max_steps=100, epsilon=1e-4)

theta[0:10] [-0.09594988  0.15150568  0.09589963 -0.12253131  0.03141645 -0.06312394
  0.06017164  0.12125044 -0.11149135 -0.26911384]
[step 1] with likelihood: -390.081717101699; theta diff:  4.383183
theta[0:10] [-0.16310458 -0.15963652 -0.07275865  0.17277371 -0.00469099  0.0624391
 -0.02150201 -0.16693785  0.06255899  0.25131659]
[step 2] with likelihood: -740.5381781512043; theta diff:  34.082963
theta[0:10] [-0.1265106   0.13262915  0.07330054 -0.1223742   0.03570405 -0.06266097
  0.00572104  0.13622498 -0.10313764 -0.25063264]
[step 3] with likelihood: -2054.7503939080684; theta diff:  1287.040308
theta[0:10] [-0.18229579 -0.1329754  -0.07293941  0.17790312 -0.0130212   0.0530392
  0.02838887 -0.17076875  0.08269547  0.26263253]
[step 4] with likelihood: -1053.7032082368542; theta diff:  62.873642
theta[0:10] [-0.11005808  0.10018839  0.07223351 -0.117917    0.0566832  -0.05012668
 -0.03948094  0.12986031 -0.1259202  -0.25793149]
[step 5] with likelihood: -1976.87260845697; thet

In [ ]:
mle_model.beta.shape

In [ ]:
norm(mle_model.theta - theta)

In [ ]:
mle_beta, mle_sigma = mle_model.copy_beta_sigma()

In [31]:
norm(mle_beta.ravel() - beta[1:].ravel())

0.008197866625751097

In [32]:
mle_beta

array([[0.2011431 , 0.19995817, 0.20191043, 0.19917045, 0.19972169,
        0.20231462, 0.20072793, 0.2006139 , 0.19902897, 0.19977722,
        0.2027578 , 0.19907222, 0.19928532, 0.19644997, 0.20039785,
        0.20252822, 0.19851476, 0.19887826, 0.199741  , 0.20045011,
        0.19983549, 0.1989687 , 0.19825504, 0.20317955, 0.19715122]])

In [33]:
norm(mle_sigma - sigma_list)

0.17668208754240278

In [34]:
mle_sigma, sigma_list

(array([0.10066939, 0.19608029, 0.29925329, 0.40004859, 0.49460891,
        0.61077638, 0.69108055, 0.78464279, 0.89756645, 1.00035397,
        1.11027765, 1.21123057, 1.31938901, 1.42742062, 1.5262268 ,
        1.6003568 , 1.72805982, 1.75367621, 1.78434146, 1.88875321]),
 array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
        1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ]))

# Compare

In [14]:
B = 100
# alpha = 1
# annotator sigma
# sigma_list = np.arange(1, M+1, 1) / 10
# parameter vector
theta = np.zeros(K * p + M)
theta[:(p * K)] = beta[1:].ravel()
theta[(p * K):] = sigma_list.reshape(-1)

RMSE_list = []

In [15]:
from model.Initial import Initial
from model.MLE import MLE

In [16]:
for seed in range(B):
    np.random.seed(seed)
    # features - X
    X = np.random.randn(N, p)  # features
    X[:, 0] = 1  # set the first columns of X to be constants
    # ty rue labels - Y
    Y = np.argmax(X.dot(np.transpose(beta)), axis=1)
    # pilot sample - X1 and Y1
    X1, X2, Y1, Y2 = train_test_split(X, Y, test_size=(N - n) / N, random_state=seed)
    # annotation task assignment - A1
    A1 = np.random.binomial(1, alpha, size=(n, M))
    # annotation probability - AP1
    AP1 = X1.dot(np.transpose(beta))
    AP1 = AP1.reshape(AP1.shape + (1,))
    AP1 = AP1 / sigma_list
    AP1 = np.exp(AP1)
    AP1_sum = AP1.sum(axis=1, keepdims=True)
    AP1 = AP1 / AP1_sum
    # annotation - AY1
    AY1 = np.zeros((n, M))
    for i in range(n):
        for m in range(M):
            prob_im = AP1[i, :, m]
            Y_im = np.argmax(np.random.multinomial(1, prob_im, 1))
            AY1[i, m] = Y_im
    AY1[A1 == 0] = -1

    # initial estimator
    initial_model = Initial(X1, AY1, A1, K)
    initial_model.opt_alg()
    rmse1 = norm(initial_model.initial_beta.ravel() - beta[1:].ravel())
    rmse2 = norm(initial_model.initial_sigma - sigma_list)

    # OS
    os_model = OS(X1, AY1, A1, K, initial_model.initial_beta, initial_model.initial_sigma)
    os_beta = os_model.one_step_update()
    rmse3 = norm(os_beta - beta[1:].ravel())

    # MLE
    mle_model = MLE(X1, AY1, A1, K)
    mle_model.GD_alg(max_steps=500, epsilon=1e-4)
    mle_beta, mle_sigma = mle_model.copy_beta_sigma()
    rmse4 = norm(mle_beta.ravel() - beta[1:].ravel())
    rmse5 = norm(mle_sigma - sigma_list)

    RMSE_list.append([rmse1, rmse2, rmse3, rmse4, rmse5])
    a = pd.DataFrame(RMSE_list)
    a.to_csv("/Users/helenology/Desktop/data1.csv")

ValueError: could not broadcast input array from shape (3,25) into shape (2,25)